In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
wordnet_lemmatizer = WordNetLemmatizer()
ps = PorterStemmer()

import string
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()
from pprint import pprint
from unidecode import unidecode
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, linear_kernel
import warnings
from tqdm._tqdm_notebook import tqdm_notebook
from scipy.spatial import distance
import numpy as np
import re
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances
from sentence_transformers import SentenceTransformer
from nltk.corpus import stopwords
stop_words = stopwords.words("english")
import tensorflow as tf
import transformers
from sent2vec.vectorizer import Vectorizer as S2vectorizer
tqdm_notebook.pandas()
warnings.filterwarnings("ignore")

#import pipelines
from transformers import pipeline
nlp2 = pipeline("question-answering")

pd.set_option('display.max_colwidth', 0)
pd.set_option('display.max_columns', 0)

/Users/gigi/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
/Users/gigi/opt/anaconda3/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [4]:
import os
os.environ['JAVAHOME'] = 'D:/Java/bin'
os.environ['STANFORD_PARSER'] = '/Applications/stanford-parser'
os.environ['STANFORD_MODELS'] = '/Applications/stanford-parser'

In [5]:
from nltk.parse.corenlp import CoreNLPParser
parser = CoreNLPParser(url='http://localhost:9000')

In [20]:
import math
import re
from collections import Counter

WORD = re.compile(r"\w+")

def get_cosine(vec1, vec2):
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])

    sum1 = sum([vec1[x] ** 2 for x in list(vec1.keys())])
    sum2 = sum([vec2[x] ** 2 for x in list(vec2.keys())])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)

    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator
    
def text_to_vector(text):
    words = WORD.findall(text)
    return Counter(words)

In [105]:
query = "What distinction is undoubtedly permissible?"
query = re.sub(r'[^\w\s]', '', query)
print("Question:", query.lower())
result = next(parser.raw_parse((query).lower()))
query_split = query.split(" ")
sub_clause_labels = ['NP', 'N', 'VP', 'PP']
new_query = ''
# print(result[0])

if query_split[0].lower() in ['what', 'when', 'why', 'how', 'where', 'which', 'who']:
        if subtree.label() in sub_clause_labels:
            new_query += ' '.join(subtree.leaves()) + " "
else:
    for subtree in result:
        if subtree.label() in sub_clause_labels:
            new_query += ' '.join(subtree.leaves()) + " "

if len(new_query) == 0:
    new_query += query.lower()
    
print("Clause:", new_query)

Question: what distinction is undoubtedly permissible
Clause: must be consulted to discover whether they allow quiet hunting or prohibit it altogether 


In [95]:
context ="Nevertheless, although a distinction between lawful and unlawful hunting is undoubtedly permissible, it is certain that a bishop can absolutely prohibit all hunting to the clerics of his diocese, as was done by synods at Milan, Avignon, LiÃ¨ge, Cologne, and elsewhere. Benedict XIV (De synodo diÅ“ces., l. II, c. x) declared that such synodal decrees are not too severe, as an absolute prohibition of hunting is more conformable to the ecclesiastical law. In practice, therefore, the synodal statutes of various localities must be consulted to discover whether they allow quiet hunting or prohibit it altogether."

In [96]:
context = context.lower()
context_split = re.split('[!#$%&\()*+-./:;<=>?@[\\]^_`{|}~]', context)
for i in range(len(context_split)):
    context_split[i] = context_split[i].strip()
    
context_split = [x for x in context_split if len(x)>0]

In [97]:
clauses = []
labels = ['SBAQ', 'S', 'SQ', 'SBAR', 'FRAG', 'SBARQ']
for cs in context_split:
    result = next(parser.raw_parse(cs))
    if result[0].label() in labels:
        clauses.append(' '.join(result[0].leaves()))
print(clauses)

['although a distinction between lawful and unlawful hunting is undoubtedly permissible', 'it is certain that a bishop can absolutely prohibit all hunting to the clerics of his diocese', 'as was done by synods at milan', 'and elsewhere', 'declared that such synodal decrees are not too severe', 'as an absolute prohibition of hunting is more conformable to the ecclesiastical law', 'the synodal statutes of various localities must be consulted to discover whether they allow quiet hunting or prohibit it altogether']


In [104]:
clauses_dictionary = {}
clause_counter = 0
np_dictionary = {}

for clause in clauses:
    result = next(parser.raw_parse(clause))
    if result[0].label() in labels:
        clause_counter += 1
        clauses_dictionary[str(clause_counter)] = {}
        clauses_dictionary[str(clause_counter)][str(clause_counter) + "_" + "clause"] = ' '.join(result[0].leaves())
        np_dictionary[str(clause_counter) + "_NVP_" + "clause"] = ''
        inner_clause1 = 0
        
        for subtree in result[0]:
            if subtree.label() in labels:
                inner_clause1 += 1
                clauses_dictionary[str(clause_counter)][str(clause_counter) + "_" + str(inner_clause1) + "_" + "clause"] = ' '.join(subtree.leaves())
                np_dictionary[str(clause_counter) + "_" + str(inner_clause1) + "_NVP_" + "clause"] = ''
                
                inner_clause2 = 0
                for subtree2 in subtree:
                    if subtree2.label() in labels:
                        inner_clause2 += 1
                        clauses_dictionary[str(clause_counter)][str(clause_counter) + "_" + str(inner_clause1) + "_" + str(inner_clause2) + "_" + "clause"] = ' '.join(subtree2.leaves())
                        np_dictionary[str(clause_counter) + "_" + str(inner_clause1) + "_" + str(inner_clause2) + "_NVP_" + "clause"] = ""
                        
                        for subtree3 in subtree2:
                            np_dictionary[str(clause_counter) + "_" + str(inner_clause1) + "_" + str(inner_clause2) + "_NVP_" + "clause"] += ' '.join(subtree3.leaves()) + " "
                    
                    elif subtree2.label() in sub_clause_labels:
                        np_dictionary[str(clause_counter) + "_" + str(inner_clause1) + "_NVP_" + "clause"] += ' '.join(subtree2.leaves()) + " "
                    
            elif subtree.label() in sub_clause_labels:
                np_dictionary[str(clause_counter) + "_NVP_" + "clause"] += ' '.join(subtree.leaves()) + " "

print("Original question:", query)
print()
question_np = new_query
question_vector = text_to_vector(question_np)
cosine_np_dict = {}
cosine_np_dict_lemm = {}

# print(clauses_dictionary)
# print()
# print(np_dictionary)
question_np_lemm = text = ' '.join([wordnet_lemmatizer.lemmatize(word) for word in question_np.split() if word not in stop_words])
question_vector_lemm = text_to_vector(question_np_lemm)

for np in np_dictionary:
    np_vector = text_to_vector(np_dictionary[np])
    cosine_np_dict[np] = get_cosine(question_vector, np_vector)
    
    np_lemm = ' '.join([wordnet_lemmatizer.lemmatize(word) for word in np_dictionary[np].split() if word not in stop_words])
    np_vector_lemm = text_to_vector(np_lemm)
    cosine_np_dict_lemm[np] = get_cosine(question_vector_lemm, np_vector_lemm)

import operator
max_clause_id, max_sim = max(cosine_np_dict.items(), key=operator.itemgetter(1))[0], max(cosine_np_dict.items(), key=operator.itemgetter(1))[1]

if max_sim >= 0.2:
    clause_id = max_clause_id[:max_clause_id.find("NVP")-1]
    first_clause_id = clause_id[:clause_id.find("_")]
    
    answer = clauses_dictionary[first_clause_id][clause_id + "_clause"]
    answer_split = answer.split(" ")
    if answer_split[0] in ["and", 'but', 'or', 'nor', 'after', 'before', 'although']:
        answer_split.pop(0)
    answer_return = " ".join(answer_split)
    print("Model's Answer:", answer_return)
else:
    print("Unable to retrieve answer")

# max_clause_id_lemm, max_sim_lemm = max(cosine_np_dict_lemm.items(), key=operator.itemgetter(1))[0], max(cosine_np_dict_lemm.items(), key=operator.itemgetter(1))[1]

# if max_sim_lemm >= 0.25:
#     clause_id_lemm = max_clause_id_lemm[:max_clause_id_lemm.find("NVP")-1]

#     answer_lemm = clauses_dictionary[clause_id_lemm][clause_id_lemm + "_clause"]
#     answer_split_lemm = answer_lemm.split(" ")
#     if answer_split_lemm[0] in ["and", 'but', 'or', 'nor', 'after', 'before', 'although']:
#         answer_split_lemm.pop(0)
#     answer_return_lemm = " ".join(answer_split_lemm)
#     print("Answer:", answer_return_lemm)
# else:
#     print("Unable to retrieve answer")


Original question: What distinction is undoubtedly permissible

Model's Answer: a distinction between lawful and unlawful hunting is undoubtedly permissible
